## Indox Retrieval Augmentation
Here, we will explore how to work with Indox Retrieval Augmentation. We are using Mistral, we should set our HUGGINGFACE_API_KEY as an environment variable.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')

### Creating an instance of IndoxTetrivalAugmentation

To effectively utilize the Indox Retrieval Augmentation capabilities, you must first create an instance of the IndoxRetrievalAugmentation class. This instance will allow you to access the methods and properties defined within the class, enabling the augmentation and retrieval functionalities.

In [3]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

### Generating response using Mistral's language models 
MistralQA class is used to handle question-answering task using Mistral's language models from HuggingFace. This instance creates HuggingFaceEmbedding class to specifying embedding model.By using UnstructuredLoadAndSplit function we can import various file types and split them into chunks.

In [4]:
from indox.llms import Mistral
from indox.embeddings import HuggingFaceEmbedding
from indox.data_loader_splitter import UnstructuredLoadAndSplit
from indox.embeddings import MistralEmbedding
mistral_qa = Mistral(api_key=MISTRAL_API_KEY)
embed_hf = HuggingFaceEmbedding(model="multi-qa-mpnet-base-cos-v1")
embed_mistral = MistralEmbedding(MISTRAL_API_KEY)
file_path = "sample.txt"
loader_splitter = UnstructuredLoadAndSplit(file_path=file_path,max_chunk_size=400)
docs = loader_splitter.load_and_chunk()


 Here ChromaVectorStore handles the storage and retrieval of vector embeddings by specifying a collection name and sets up a vector store where text embeddings can be stored and queried.

In [5]:
from indox.vector_stores import ChromaVectorStore
db = ChromaVectorStore(collection_name="sample",embedding=embed_mistral)
indox.connect_to_vectorstore(vectorstore_database=db)

### load and preprocess data
This part of code demonstrates how to load and preprocess text data from a file, split it into chunks, and store these chunks in the vector store that was set up previously.

In [6]:
from indox.data_loader_splitter import UnstructuredLoadAndSplit
loader_splitter = UnstructuredLoadAndSplit(file_path=file_path,max_chunk_size=400)
docs = loader_splitter.load_and_chunk()

In [7]:
docs

[Document(page_content="The wife of a rich man fell sick, and as she felt that her end\n\nwas drawing near, she called her only daughter to her bedside and\n\nsaid, dear child, be good and pious, and then the\n\ngood God will always protect you, and I will look down on you\n\nfrom heaven and be near you. Thereupon she closed her eyes and\n\ndeparted. Every day the maiden went out to her mother's grave,", metadata={'filename': 'sample.txt', 'filetype': 'text/plain', 'last_modified': '2024-05-30T13:53:09'}),
 Document(page_content='and wept, and she remained pious and good. When winter came\n\nthe snow spread a white sheet over the grave, and by the time the\n\nspring sun had drawn it off again, the man had taken another wife.\n\nThe woman had brought with her into the house two daughters,\n\nwho were beautiful and fair of face, but vile and black of heart.\n\nNow began a bad time for the poor step-child. Is the stupid goose', metadata={'filename': 'sample.txt', 'filetype': 'text/plain',

In [8]:
indox.store_in_vectorstore(docs=docs)

### Retrieve relevant information and generate an answer
The main purpose of these lines is to perform a query on the vector store to retrieve the most relevant information (top_k=5) and generate an answer using the language model.

In [9]:
query = "How cinderella reach her happy ending?"
retriever = indox.QuestionAnswer(vector_database=db,llm=mistral_qa,top_k=5)

invoke(query) method sends the query to the retriever, which searches the vector store for relevant text chunks and uses the language model to generate a response based on the retrieved information.
Context property retrieves the context or the detailed information that the retriever used to generate the answer to the query. It provides insight into how the query was answered by showing the relevant text chunks and any additional information used.

In [10]:
retriever.invoke(query)

"Based on the context provided, Cinderella's happy ending involved the king's son falling in love with her at a festival that lasted three days. Despite her step-sisters' and step-mother's attempts to prevent her from attending, Cinderella was able to go with the help of a bird who granted her wishes. At the festival, the king's son only danced with Cinderella and tried to follow her when she left, but she managed to escape. However, the king's son had smeared the staircase with pitch, and Cinderella's glass slipper got stuck in it. The king's son used the slipper to find Cinderella, and they presumably lived happily ever after.\n\nIt's worth noting that the context provided only covers part of the Cinderella story, and some details may vary depending on the version of the tale. However, based on the information given, this is the most likely sequence of events leading to Cinderella's happy ending."

In [11]:
retriever.context

['by the hearth in the cinders. And as on that account she always\n\nlooked dusty and dirty, they called her cinderella.\n\nIt happened that the father was once going to the fair, and he\n\nasked his two step-daughters what he should bring back for them.\n\nBeautiful dresses, said one, pearls and jewels, said the second.\n\nAnd you, cinderella, said he, what will you have. Father',
 "to appear among the number, they were delighted, called cinderella\n\nand said, comb our hair for us, brush our shoes and fasten our\n\nbuckles, for we are going to the wedding at the king's palace.\n\nCinderella obeyed, but wept, because she too would have liked to\n\ngo with them to the dance, and begged her step-mother to allow\n\nher to do so. You go, cinderella, said she, covered in dust and",
 "danced with her only, and if any one invited her to dance, he said\n\nthis is my partner.\n\nWhen evening came, cinderella wished to leave, and the king's\n\nson was anxious to go with her, but she escaped fro

### With AgenticRag

AgenticRag stands for Agentic Retrieval-Augmented Generation. This concept combines retrieval-based methods and generation-based methods in natural language processing (NLP). The key idea is to enhance the generative capabilities of a language model by incorporating relevant information retrieved from a database or a vector store. 
 AgenticRag is designed to provide more contextually rich and accurate responses by utilizing external knowledge sources. It retrieves relevant pieces of information (chunks) from a vector store based on a query and then uses a language model to generate a comprehensive response that incorporates this retrieved information.

In [ ]:
agent = indox.AgenticRag(llm=mistral_qa,vector_database=db,top_k=5)
agent.run(query)